In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: nltk in c:\users\91620\appdata\local\programs\python\python312\lib\site-packages (3.9.1)




[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\91620\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [ ]:
# train.csv: A full training dataset with the following attributes:

# id: unique id for a news article
# title: the title of a news article
# author: author of the news article
# text: the text of the article; could be incomplete
# label: a label that marks the article as potentially unreliable
# 1: unreliable
# 0: reliable
# test.csv: A testing training dataset with all the same attributes at train.csv without the label.

# submit.csv: A sample submission that you can



In [40]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91620\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
news_dataset = pd.read_csv('train.csv')

In [6]:
news_dataset.shape

(20800, 5)

In [7]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
news_dataset=news_dataset.fillna('')

In [10]:
news_dataset['content']=news_dataset['author'] +' '+news_dataset['title'] 

In [11]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [12]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
X = news_dataset.drop(columns='label',axis=1)
Y=news_dataset['label']

In [14]:
X.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


In [15]:
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [16]:
# now apply stemming to it 
port_stem = PorterStemmer()

In [17]:
def stemming(content):
    stemmed_content = re.sub('^a-zA-Z',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split();
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [18]:
news_dataset['content']= news_dataset['content'].apply(stemming)

In [19]:
news_dataset['content'].head()

0    darrel lucu hous dem aide: didn’t even see com...
1    daniel j. flynn flynn: hillari clinton, big wo...
2              consortiumnews.com truth might get fire
3    jessica purkiss 15 civilian kill singl us airs...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object

In [20]:
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [21]:
print(X)

['darrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'consortiumnews.com truth might get fire' ...
 'michael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansari nato, russia hold parallel exercis balkan'
 'david swanson keep f-35 aliv']


In [22]:
print(Y)

[1 0 1 ... 0 1 1]


In [23]:
Y.shape

(20800,)

In [24]:
X.shape

(20800,)

In [25]:
# converting the textual to numerical data 
vectorizer= TfidfVectorizer()
vectorizer.fit(X)
X= vectorizer.transform(X)


In [26]:
print(X)

  (0, 19097)	0.27315635150958634
  (0, 16473)	0.23676064517956458
  (0, 11072)	0.33384522056560495
  (0, 10747)	0.2682220926318631
  (0, 9692)	0.22757176689298136
  (0, 8832)	0.205341824533184
  (0, 6433)	0.2142258791026174
  (0, 5256)	0.27468869329117757
  (0, 4995)	0.2512923264945339
  (0, 4763)	0.3304457115379666
  (0, 3952)	0.22664699692052692
  (0, 3403)	0.3375689613898566
  (0, 809)	0.3646500188253279
  (1, 20416)	0.29951908908156866
  (1, 8608)	0.19815023888659125
  (1, 7101)	0.711483310803025
  (1, 4728)	0.26268668599849243
  (1, 3778)	0.19062686807106288
  (1, 3100)	0.3870784468942128
  (1, 2713)	0.15460118725006144
  (1, 2258)	0.2928176012009572
  (2, 19015)	0.41491113753784553
  (2, 11878)	0.49151393723208897
  (2, 7650)	0.34605253138342823
  (2, 6968)	0.39293503470255664
  :	:
  (20797, 15142)	0.2718547014681534
  (20797, 14906)	0.24707379158386517
  (20797, 12685)	0.08013411384942215
  (20797, 11844)	0.17405416749037161
  (20797, 11767)	0.2945753459972408
  (20797, 11179)	

In [34]:
# splitting the dataset to train and testing data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)
model = LogisticRegression()
model.fit(X_train,Y_train)
# accuracy score on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)
print('Accuracy score in the training data :', training_data_accuracy)
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)
print('Accuracy score in the testing data :', test_data_accuracy)
x_news= X_test[0]
prediction=model.predict(x_news)
print(prediction)
if(prediction == 0):
 print('The news is real')
else:
 print('The news is fake')


Accuracy score in the training data : 0.9865384615384616
Accuracy score in the testing data : 0.9774038461538461
[1]
The news is fake


In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, Y_train)

# Make predictions on the training data
X_train_prediction = model.predict(X_train)

# Calculate training data accuracy using R-squared
training_data_r2 = r2_score(Y_train, X_train_prediction)
print('R-squared score on the training data:', training_data_r2)

# Make predictions on the test data
X_test_prediction = model.predict(X_test)

# Calculate test data accuracy using R-squared
test_data_r2 = r2_score(Y_test, X_test_prediction)
print('R-squared score on the testing data:', test_data_r2)

# Predict a single sample (e.g., the first test sample)
x_news = X_test[0].reshape(1, -1)  # Reshape to match the input shape
prediction = model.predict(x_news)
print('Prediction for X_test[0]:', prediction)

# Interpretation
# print(f'The predicted value for x_news ({x_news.flatten()}) is approximately {prediction[0]:.2f}')

R-squared score on the training data: 0.9948028174562143
R-squared score on the testing data: 0.40331019058950135
Prediction for X_test[0]: [0.95482303]


In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# Create the KNN model
# n_neighbors specifies the number of neighbors to consider
model = KNeighborsClassifier(n_neighbors=3)

# Train the model
model.fit(X_train, Y_train)

# Predict on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on the training data:', training_data_accuracy)

# Predict on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score on the testing data:', test_data_accuracy)

# Predict for a single instance
x_news = X_test[0].reshape(1, -1)  # Reshape to match the input shape
prediction = model.predict(x_news)
print('Prediction for X_test[0]:', prediction)

# Interpretation
if prediction == 0:
    print('The news is real')
else:
    print('The news is fake')

Accuracy score on the training data: 0.9685697115384615
Accuracy score on the testing data: 0.9221153846153847
Prediction for X_test[0]: [1]
The news is fake


In [ ]:
# Split the data into training and testing sets
X_sparse=X;
X = X_sparse.toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create the Naive Bayes model
model = GaussianNB()

# Train the model
model.fit(X_train, Y_train)

# Predict on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on the training data:', training_data_accuracy)

# Predict on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score on the testing data:', test_data_accuracy)

# Predict for a single instance
x_news = X_test[0].reshape(1, -1)  # Reshape to match the input shape
prediction = model.predict(x_news)
print('Prediction for X_test[0]:', prediction)

# Interpretation
if prediction == 0:
    print('The news is real')
else:
    print('The news is fake')

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create the Multinomial Naive Bayes model
model = MultinomialNB()

# Train the model
model.fit(X_train, Y_train)

# Predict on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on the training data:', training_data_accuracy)

# Predict on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score on the testing data:', test_data_accuracy)

# Predict for a single instance
x_news = X_test[0]
prediction = model.predict(x_news)
print('Prediction for X_test[0]:', prediction)

# Interpretation
if prediction == 0:
    print('The news is real')
else:
    print('The news is fake')

Accuracy score on the training data: 0.9792668269230769
Accuracy score on the testing data: 0.95
Prediction for X_test[0]: [1]
The news is fake


In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert sparse matrix to dense array (RandomForest requires dense input)
X_train = X_train.toarray()
X_test = X_test.toarray()

# Create the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, Y_train)

# Predict on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on the training data:', training_data_accuracy)

# Predict on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score on the testing data:', test_data_accuracy)

# Predict for a single instance
x_news = X_test[0].reshape(1, -1)  # Reshape to match the input shape
prediction = model.predict(x_news)
print('Prediction for X_test[0]:', prediction)

# Interpretation
if prediction == 0:
    print('The news is real')
else:
    print('The news is fake')

MemoryError: Unable to allocate 693. MiB for an array with shape (4160, 21822) and data type float64

In [41]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create the SVM model using LinearSVC
model = LinearSVC(random_state=42)
model.fit(X_train, Y_train)

# Predict on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on the training data:', training_data_accuracy)

# Predict on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score on the testing data:', test_data_accuracy)

# Predict for a single instance
x_news = X_test[0]  # Sparse format
prediction = model.predict(x_news)
print('Prediction for X_test[0]:', prediction)

# Interpretation
if prediction == 0:
    print('The news is real')
else:
    print('The news is fake')

Accuracy score on the training data: 0.9995793269230769
Accuracy score on the testing data: 0.9896634615384615
Prediction for X_test[0]: [1]
The news is fake


C:\Users\91620\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [44]:
# Expanded dataset with more diverse examples
documents = [
    "John Doe - Breaking News: The sky is blue today",
    "Jane Smith - Bright Future Ahead: The sun is shining brightly",
    "Emily Davis - AI technology is advancing rapidly",
    "Michael Lee - Sports Update: The local team wins the championship",
    "Unknown Author - Aliens landed on Earth in the middle of the night",
    "Sarah Wilson - Heavy rains expected tomorrow",
    "Alex Johnson - Stock prices are soaring high",
    "Fake News Bot - The moon is made of cheese"
]
labels = [0, 0, 0, 0, 1, 0, 0, 1]

# Train the model again
X = vectorizer.fit_transform(documents)
Y = labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model.fit(X_train, Y_train)

# Take user input
author_name = input("Enter the author name: ")
title = input("Enter the title of the news: ")
user_input = f"{author_name} - {title}".lower()
user_input_vectorized = vectorizer.transform([user_input])

# Predict and display
prediction = model.predict(user_input_vectorized)
if prediction[0] == 0:
    print("\nThe news is REAL.")
else:
    print("\nThe news is FAKE.")


C:\Users\91620\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Enter the author name:   Emily Davis  
Enter the title of the news:  AI is revolutionizing the future of work 


The news is fake.
